In [4]:
import json
import sqlite3
import pandas as pd
import config
import numpy as np
import requests

from sqlalchemy import create_engine
engine = create_engine('postgresql://'+config.db['user']+':'+config.db['password']+config.db['url'])
#f = open('./data/NVDRstock-trading31-01-23.json')
#data = json.load(f)
#data = pd.read_json('./data/NVDRstock-trading31-01-23.json')
sSource = "file"
if sSource=="file":
    with open('./data/Pricecomposition02-02-23.json','r',encoding='utf-8') as f:
        data = json.loads(f.read())
elif sSource=="api":
    r = requests.get('https://www.set.or.th/api/set/index/set100/composition?lang=en')
    data = r.json()
    if r.status_code!=200:
        exit

print(data['indexInfos'][0]['marketDateTime'][:10])
#cdate = pd.DataFrame(data).iloc[0].astype(str).str[:10]['date'], record_prefix=
#d1.info()
#print(cdate)
df = pd.json_normalize(data['composition']['stockInfos']) 
df.drop(['average', 'floor','ceiling', 'trVolume','trValue', 'aomVolume','aomValue', 'bids','offers'], axis=1, inplace=True)
df.rename(columns = {'symbol': 'series', 'last': 'close', 'percentChange': 'pchange','totalVolume': 'vol','totalValue': 'value'}, inplace = True)
df.drop(df.iloc[:, 11:],axis = 1, inplace=True)
df['trddate'] = data['indexInfos'][0]['marketDateTime'][:10] #pd.to_datetime('today').date() - pd.Timedelta("1 day")
print(df['trddate'][0])
a = [0]
df = df[~df['close'].isin(a)]
df.info()
print(df)
# df['composition.stockInfos'].info()
# print(df['composition.stockInfos'])


2023-02-02
2023-02-05
<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   series   100 non-null    object 
 1   sign     100 non-null    object 
 2   prior    100 non-null    float64
 3   close    100 non-null    float64
 4   open     100 non-null    float64
 5   high     100 non-null    float64
 6   low      100 non-null    float64
 7   change   100 non-null    float64
 8   pchange  100 non-null    float64
 9   vol      100 non-null    float64
 10  value    100 non-null    float64
 11  trddate  100 non-null    object 
dtypes: float64(9), object(3)
memory usage: 10.2+ KB
    series sign   prior   close    open    high     low  change   pchange  \
0      AAV         3.08    3.08    3.10    3.14    3.08    0.00  0.000000   
1      ACE         2.58    2.56    2.58    2.58    2.56   -0.02 -0.775194   
2   ADVANC       198.00  197.50  198.50  200.00  197.00   

In [2]:
print(df)
sql = "select * from sprice where trddate = '"+ df['trddate'].astype(str)[0] + "'"
print(sql)
rst = pd.read_sql(sql, con=engine)
print(len(rst.axes[0]))
if len(rst.axes[0]) == 0:df.to_sql(name='sprice', con=engine, if_exists='append')
p2 = pd.read_sql("select * from sprice where trddate = '"+ pd.to_datetime('today').strftime('%Y-%m-%d') + "'", con=engine)
print(p2)
len(df)

    series sign   prior   close    open    high     low  change   pchange  \
0      AAV         3.06    3.10    3.08    3.10    3.06    0.04  1.307190   
1      ACE         2.54    2.56    2.54    2.58    2.54    0.02  0.787402   
2   ADVANC       198.50  197.00  198.50  198.50  197.00   -1.50 -0.755668   
3    AMATA        20.40   20.20   20.50   20.50   20.20   -0.20 -0.980392   
4      AOT        73.75   73.75   74.00   74.25   73.75    0.00  0.000000   
..     ...  ...     ...     ...     ...     ...     ...     ...       ...   
95    TRUE         4.88    4.86    4.90    4.90    4.84   -0.02 -0.409836   
96     TTB         1.45    1.45    1.45    1.45    1.44    0.00  0.000000   
97      TU        16.10   16.20   16.10   16.50   16.10    0.10  0.621118   
98     VGI         5.05    5.00    5.05    5.05    5.00   -0.05 -0.990099   
99     WHA         3.98    3.92    3.96    4.00    3.88   -0.06 -1.507538   

           vol         value     trddate  
0   24126162.0  7.449050e+07  20

100

In [ ]:
#print(df)
df.rename(columns = {'date': 'trddate', 'buyValue': 'buy', 'sellValue': 'sell', 'netValue': 'net', 'totalValue': 'total', 'percentValue': 'percent', 'underlyingValue':'trdvalue'}, inplace = True)
df['ft']=df['total']/df['trdvalue']*100
df['ft'].round(2)
#df.info()
print(df)
#r.json()
#print(r.status_code)
#df = pd.json_normalize(r.json(), record_path =['nvdrTradings'])

#print(df)
df.rename(columns = {'date': 'trddate', 'buyValue': 'buy', 'sellValue': 'sell', 'netValue': 'net', 'totalValue': 'total', 'percentValue': 'percent', 'underlyingValue':'trdvalue'}, inplace = True)
df['ft']=df['total']/df['trdvalue']*100
df['ft'].round(2)
#df.info()
print(df)